In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import tensorflow as tf

import tensorflow_hub as hub    # imports the weights, call of bert

from tensorflow.python.keras import layers
import bert
import nltk
nltk.download('punkt')  # Download the Punkt tokenizer model

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\henry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
# Read the dataframe from createDebate.xlsx with headers
df_createDebate = pd.read_excel('createDebate.xlsx', header=0)

unknown                       3051
prefers strict gun control    3051
opposes strict gun control    3051
undecided                     3051
other                         3051
Name: stance, dtype: int64

In [35]:
columns_to_keep = ['stance', 'text']
data = df_createDebate.filter(columns_to_keep)

In [36]:
def clean_text(text):
    """
    Preprocess text input, remove insignificant parts
    :param text: text input
    :return: clean text
    """
    # Keep only the plain text and meaningful punctuations.
    text = re.sub(r"[^a-zA-Z.!?']", ' ', text)
    # Remove additional spaces.
    text = re.sub(r" +", ' ', text)
    return text

def sep_sentences(text):
    """
    Seperate sentences
    :param text: the plain text input
    :return: a list of sentences from the text
    """
    sentences = sent_tokenize(text)
    return sentences

# text = "Hello** Jona, this is Michael! You're so nice. I hate going out with Your friends."
# text = clean_text(text)
# li = sep_sentences(text)
# print(li)

In [37]:
data_clean = [clean_text(text) for text in data.text]    # Clean the text
data_sent = [sep_sentences(text) for text in data_clean]    # Seperate the sentences

In [38]:
data_stance = data.stance

In [39]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [40]:
def encode_sentence(sent):
    """
    Encode the sentence. Add [CLS] and [SEP] labels to the tokenized sentence just so BERT would understand.
    :param sent: the sentence to be encoded
    :return: encoded sentence
    """
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [41]:
# encode every sentence from data_sent
data_inputs = [
    [encode_sentence(sent) for sent in text] for text in data_sent
]

In [34]:
# Testing
# print(data_inputs[1][1])

['[CLS]', 'guns', 'provide', 'a', 'trigger', 'for', 'any', 'violent', 'activity', 'in', 'the', 'society', 'like', 'the', 'virginia', 'university', 'incident', '.', '[SEP]']


In [42]:
# Author: Martin J. et al.
# Link: https://colab.research.google.com/drive/1jMin0iXmW4ZrSlwjhj2xJAiCrUMOLsqg?usp=sharing
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # convert 1 into 0 and vice versa
    return seg_ids

In [43]:
# Data output (Maybe?)
stance_output = [encode_sentence(sentence) for sentence in data_stance]

In [46]:
# Establish the length-data relationship for shuffling
data_len = [[text, stance_output[i], len(text)] for i, text in enumerate(data_inputs)]
random.shuffle(data_len)

# Sort accroding to how many sentences are there in each text string.
data_len.sort(key=lambda x: x[2])
# Only take the texts with 2 or more sentences into account.
sorted_all = [(sent_lab[0], sent_lab[1]) for sent_lab in data_len if sent_lab[2] >= 2]

In [47]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, output_types=(tf.int32, tf.int32))

In [48]:
next(iter(all_dataset))

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element that could not be converted to the expected type. The expected type was int32, but the yielded element was [['[CLS]', 'guns', 'should', 'be', 'banned', '.', '[SEP]'], ['[CLS]', 'guns', 'are', 'meant', 'to', 'protect', 'not', 'kill', 'but', 'in', 'the', 'last', 'few', 'days', 'we', 'are', 'seeing', 'people', 'killing', 'each', 'other', '[SEP]']].
Traceback (most recent call last):

  File "C:\Program Files\Python310\lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 214, in generator_py_func
    script_ops.FuncRegistry._convert(  # pylint: disable=protected-access

  File "C:\Program Files\Python310\lib\site-packages\tensorflow\python\ops\script_ops.py", line 226, in _convert
    result = np.asarray(value, dtype=dtype, order="C")

ValueError: setting an array element with a sequence. The requ